# Run PncA WandB Sweep

In [1]:
from IPython.display import display
import os

if "SSH_CONNECTION" in os.environ:
    display("Running via SSH")
else:
    display("Running locally")
    
import sys
import os

path = os.path.join('..', '/Users/dylandissanayake/Desktop/DPhil/Comp Disc/Repositories/TB-PNCA-GNN') if "SSH_CONNECTION" not in os.environ else os.path.join('..', '/mnt/alphafold-volume-1/dylan2/repos/tb-pnca-gnn')
if path not in sys.path:
    sys.path.append(os.path.abspath(path))

import datetime
import random

import numpy as np
import pandas as pd
import pickle as pkl

import torch
from torch_geometric.data import Data

import wandb

import warnings
warnings.filterwarnings('ignore')

from src import run_model, protein_graph, gcn_model, evaluation

%load_ext autoreload
%autoreload 2

%aimport src

torch.cuda.is_available()

'Running via SSH'

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


True

In [2]:
with open('datasets/singletons_af_graph_dict.pkl', 'rb') as f:
    graph_dict = pkl.load(f)

In [3]:
len(graph_dict['train']) + len(graph_dict['test'])

664

### Set Up Params and Sweep Config

In [4]:
seed = 42
np.random.seed(seed)
random.seed(seed)

# logging params (only used for wandb metrics)
n_samples = len(graph_dict['train']) + len(graph_dict['test'])
# cutoff_distance = 6.3  

# gcn params
num_node_features = 16
batch_size = 256
# hidden_channels = 64
# learning_rate = 0.001
# wd = 5e-5
epochs = 400

wt_seq = 'MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVATKDFHIDPGDHFSGTPDYSSSWPPHCVSGTPGADFHPSLDTSAIEAVFYKGAYTGAYSGFEGVDENGTPLLNWLRQRGVDEVDVVGIATDHCVRQTAEDAVRNGLATRVLVDLTAGVSADTTVAALEEMRTASVELVCS'

In [5]:
# First sweep:

sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'Test Accuracy',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'hidden_channels': {
        'values': [32, 64, 128, 256]
        },
    'weight_decay': {
        'distribution': 'log_uniform_values',
        'min': 1e-8,
        'max': 1e-2
        },
    'dropout': {
          'values': [0.2, 0.4, 0.5, 0.6, 0.8]
        },
    'cutoff_distance': {
        'distribution': 'uniform',
        'min': 4,
        'max': 12 
        },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-8,
        'max': 1e-1
      },
    }

sweep_config['parameters'] = parameters_dict

# # Second / third sweep:

# parameters_dict.update({
#     'hidden_channels': {
#         'values': [64, 128, 192, 256, 320, 384]
#         },
#     'weight_decay': {
#         'distribution': 'log_uniform_values',
#         'min': 5e-7, 
#         'max': 1e-2
#         },
#     'dropout': {
#           'values': [0.4, 0.5, 0.6, 0.8]
#         },
#     'learning_rate': {
#         'distribution': 'log_uniform_values',
#         'min': 1e-5,
#         'max': 1e-2 
#         },
#     })

# # Fourth / Fifth sweep:

# parameters_dict.update({
#     'hidden_channels': {
#         'values': [128, 192, 256, 320]
#         },
#     'weight_decay': {
#         'distribution': 'log_uniform_values',
#         'min': 1e-5,
#         'max': 1e-3
#         },
#     'dropout': {
#           'values': [0.4, 0.5, 0.6, 0.8]
#         },
#     'learning_rate': {
#         'distribution': 'log_uniform_values',
#         'min': 5e-5,
#         'max': 5e-2
#         },
#     })

# # Sixth sweep:

# parameters_dict.update({
#     'learning_rate': {
#         'distribution': 'log_uniform_values',
#         'min': 1e-6,
#         'max': 5e-3 
#         }
#     })

# # Seventh sweep:

# parameters_dict.update({
#     'learning_rate': {
#         'distribution': 'log_uniform_values',
#         'min': 5e-6,
#         'max': 5e-4 
#         }
#     })

# # Eight sweep w cutoff varying:

# parameters_dict.update({
#     'learning_rate': {
#         'distribution': 'log_uniform_values',
#         'min': 1e-15,
#         'max': 1e-3 
#         },
#         'cutoff_distance': {
#         'distribution': 'uniform',
#         'min': 5.5,
#         'max': 14
#         },
#     })



In [6]:
import pprint
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'Test Accuracy'},
 'parameters': {'cutoff_distance': {'distribution': 'uniform',
                                    'max': 12,
                                    'min': 4},
                'dropout': {'values': [0.2, 0.4, 0.5, 0.6, 0.8]},
                'hidden_channels': {'values': [32, 64, 128, 256]},
                'learning_rate': {'distribution': 'log_uniform_values',
                                  'max': 0.1,
                                  'min': 1e-08},
                'weight_decay': {'distribution': 'log_uniform_values',
                                 'max': 0.01,
                                 'min': 1e-08}}}


### Define Training Loop

In [7]:
project = "pnca-af-singletons-sweep-w-mutation-feats"

In [8]:
# sweep_id = wandb.sweep(sweep_config, project=project)
sweep_id = 'x0k5kbpt'

In [9]:
# # Run a single run

# model = run_model.pnca_GCN_vary_graph(
#             self_loops = False,
#             cutoff_distance = 4,
#             edge_weight_func = '1-(dist/cutoff)',
#             batch_size = batch_size,
#             num_node_features = num_node_features,
#             hidden_channels = 64,
#             learning_rate = 1e-5,
#             wd = 1e-5,
#             dropout = 0.5,
#             lr_scheduling=False,
#             epochs = 50,
#             graph_dict= graph_dict,
#             normalise_ews=True,
#             # wandb_params={
#             #     'use_wandb': False,
#             #     'sweep': True
#             # }
#         )

In [10]:
def sweep_run():

    with wandb.init() as run:
        config = run.config

        model = run_model.pnca_GCN_vary_graph(
            self_loops = False,
            cutoff_distance = config.cutoff_distance,
            edge_weight_func = '1-(dist/cutoff)',
            batch_size = batch_size,
            num_node_features = num_node_features,
            hidden_channels = config.hidden_channels,
            learning_rate = config.learning_rate,
            wd = config.weight_decay,
            dropout = config.dropout,
            lr_scheduling=False,
            epochs = epochs,
            graph_dict= graph_dict,
            normalise_ews=True,
            wandb_params={
                'use_wandb': False,
                'sweep': True
            }
        )

        # os.makedirs(f'saved_models/carter_ds_aug/{project}/{sweep_id}', exist_ok=True)
        
        # torch.save(model, f'saved_models/carter_ds_aug/{project}/{sweep_id}/{run.name}')

In [11]:
wandb.agent(sweep_id, sweep_run, project = project, count=100)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Agent Starting Run: 8cg6sd2f with config:
wandb: 	cutoff_distance: 7.503507123483934
wandb: 	dropout: 0.5
wandb: 	hidden_channels: 256
wandb: 	learning_rate: 6.751133651144298e-05
wandb: 	weight_decay: 0.000151560868497435
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: dyl

Adjusting edge index and attaching edge weights for cutoff distance 7.503507123483934
Using CUDA
Early stopping enabled. Patience: 20. Min Delta: 0.
Epoch: 010, Train Acc: 0.6121, Test Acc: 0.5750, Train Loss: 0.6495, Test Loss: 0.6641
Epoch: 020, Train Acc: 0.6358, Test Acc: 0.6200, Train Loss: 0.6399, Test Loss: 0.6397
Epoch: 030, Train Acc: 0.6358, Test Acc: 0.6200, Train Loss: 0.6380, Test Loss: 0.6418
Epoch: 040, Train Acc: 0.6358, Test Acc: 0.6200, Train Loss: 0.6303, Test Loss: 0.6435
Epoch: 050, Train Acc: 0.6358, Test Acc: 0.6200, Train Loss: 0.6252, Test Loss: 0.6370
Epoch: 060, Train Acc: 0.6358, Test Acc: 0.6200, Train Loss: 0.6215, Test Loss: 0.6357
Epoch: 070, Train Acc: 0.6379, Test Acc: 0.6150, Train Loss: 0.6193, Test Loss: 0.6321
Epoch: 080, Train Acc: 0.6422, Test Acc: 0.6150, Train Loss: 0.6154, Test Loss: 0.6318
Epoch: 090, Train Acc: 0.6401, Test Acc: 0.6150, Train Loss: 0.6113, Test Loss: 0.6252
Epoch: 100, Train Acc: 0.6444, Test Acc: 0.6100, Train Loss: 0.6061,

Test Accuracy,▁▅▃▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▆▆▆▆▇▆▇▇▇▇▇██▇█▇▇█
Test F1,▁█████████████████████████████████████▇▇
Test Loss,█▆▇▅▅▅▅▅▄▄▄▄▄▄▄▄▄▄▃▄▃▃▂▃▃▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁
Test Sensitivity,▁█████████████████████▇▇▇▇▇▇▆▇▇▆▆▆▆▆▆▆▆▅
Test Specificity,█▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▄▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆
Train Accuracy,▁▅▃▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▆▆▆▇▆▆▇▇▇▇▇▇▇█▇██▇
Train F1,▁████████████████████████████████▇▇████▇
Train Loss,█▆▆▅▅▆▅▅▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁
Train Sensitivity,▁▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▇▆▆▆▆▇▆▆▆▆▆▆▆▆▅
Train Specificity,█▂▁▂▂▂▂▃▃▂▃▂▃▃▃▃▃▃▃▃▃▃▄▃▄▄▄▄▅▄▄▄▅▅▅▅▅▅▆▆
Test Accuracy,0.665


wandb: Agent Starting Run: 2rsrzq7r with config:
wandb: 	cutoff_distance: 8.554294805728848
wandb: 	dropout: 0.8
wandb: 	hidden_channels: 256
wandb: 	learning_rate: 1.9446102773579997e-05
wandb: 	weight_decay: 8.66146532859258e-08
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Adjusting edge index and attaching edge weights for cutoff distance 8.554294805728848
Using CUDA
Early stopping enabled. Patience: 20. Min Delta: 0.
Epoch: 010, Train Acc: 0.6358, Test Acc: 0.6200, Train Loss: 0.6415, Test Loss: 0.6546
Epoch: 020, Train Acc: 0.6358, Test Acc: 0.6200, Train Loss: 0.7059, Test Loss: 0.6683
Epoch: 030, Train Acc: 0.6358, Test Acc: 0.6200, Train Loss: 0.7005, Test Loss: 0.6730
20 epochs passed without 0 test loss improvement. 
Early stopping triggered.


...

Adjusting edge index and attaching edge weights for cutoff distance 11.34701165656125
Using CUDA
Early stopping enabled. Patience: 20. Min Delta: 0.
Epoch: 010, Train Acc: 0.4461, Test Acc: 0.4450, Train Loss: 0.7572, Test Loss: 0.7766
Epoch: 020, Train Acc: 0.5065, Test Acc: 0.5200, Train Loss: 0.7537, Test Loss: 0.7204
Epoch: 030, Train Acc: 0.6595, Test Acc: 0.6450, Train Loss: 0.6262, Test Loss: 0.6060
Epoch: 040, Train Acc: 0.6336, Test Acc: 0.6200, Train Loss: 0.6454, Test Loss: 0.6655
Epoch: 050, Train Acc: 0.6789, Test Acc: 0.6550, Train Loss: 0.5780, Test Loss: 0.5865
Epoch: 060, Train Acc: 0.6595, Test Acc: 0.6400, Train Loss: 0.5917, Test Loss: 0.6150
Epoch: 070, Train Acc: 0.6703, Test Acc: 0.6000, Train Loss: 0.5881, Test Loss: 0.6407
Epoch: 080, Train Acc: 0.6961, Test Acc: 0.7050, Train Loss: 0.5409, Test Loss: 0.5754
Epoch: 090, Train Acc: 0.7177, Test Acc: 0.6400, Train Loss: 0.5468, Test Loss: 0.5994
Epoch: 100, Train Acc: 0.7069, Test Acc: 0.6950, Train Loss: 0.5390,

Test Accuracy,▅▄▂▁▂▂▃▇▅▇▅▅▅▅▅▅▆▆▆▅▆▅▅▆▆▇█▆▆▅█▇▆▇▆█▆▆▇▇
Test F1,██▁▁▁▁▂▇▇▇██████████████████▇▇█████████▇
Test Loss,▇█▃▃▂▃▂▂▁▂▁▂▂▂▂▂▁▁▁▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test Sensitivity,██▁▁▁▁▁▅▆▅██████▇▆██▇██▇▆▆▅▆▆▆▆▆▇▆▆▆▇▇▆▅
Test Specificity,▂▁█▇███▆▄▇▂▂▂▂▂▂▄▅▃▂▃▂▂▄▅▆▆▅▄▄▆▅▄▅▅▆▃▄▅▆
Train Accuracy,▆▅▁▁▂▂▂▆▆▆▆▆▆▆▆▆▇▇▆▆▇▆▆▇█▇▇█████████▇███
Train F1,██▁▂▂▁▂▇▇▇███████████████▇▇█████████████
Train Loss,█▅▄▂▂▂▂▂▂▂▁▂▁▁▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Sensitivity,▇█▁▂▁▁▁▅▆▅▇▇▇█▇█▇▆▇█▇██▇▇▆▅▇▇▇▆▆▇▆▇▆▇▇▆▆
Train Specificity,▂▁█▇███▆▄▆▃▃▃▂▃▂▄▅▃▂▃▃▃▄▅▆▆▅▅▅▆▆▅▆▅▆▄▄▆▆
Test Accuracy,0.68


wandb: Agent Starting Run: m838f52b with config:
wandb: 	cutoff_distance: 11.341289155058512
wandb: 	dropout: 0.6
wandb: 	hidden_channels: 256
wandb: 	learning_rate: 7.093448676343721e-07
wandb: 	weight_decay: 8.240232072745793e-05
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Adjusting edge index and attaching edge weights for cutoff distance 11.341289155058512


wandb: Ctrl + C detected. Stopping sweep.
